In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 16512, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 16512 (delta 41), reused 49 (delta 15), pack-reused 16408
Receiving objects: 100% (16512/16512), 15.12 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (11306/11306), done.


In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="QMVflpfIvOW0u1pcfDVI")
project = rf.workspace("traffic-light-detection-qsrxn").project("traffic-light-oq7uj")
version = project.version(25)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# 데이터셋 폴더 경로 설정
dataset_path = "/content/yolov5/dataset"

# data.yaml 파일 경로
data_yaml_path = dataset_path + "/data.yaml"

# data.yaml 파일 내용 확인
!cat {data_yaml_path}

names:
- green
- red
- yellow
nc: 3
roboflow:
  license: CC BY 4.0
  project: traffic-light-oq7uj
  url: https://universe.roboflow.com/traffic-light-detection-qsrxn/traffic-light-oq7uj/dataset/25
  version: 25
  workspace: traffic-light-detection-qsrxn
test: ../test/images
train: traffic-light-25/train/images
val: traffic-light-25/valid/images


In [ ]:
%cd /
from glob import glob
train_img_list = glob('/content/yolov5/dataset/train/images/*.jpg')
print(len(train_img_list))

/
1767


In [ ]:
%cd /
from glob import glob
valid_img_list = glob('/content/yolov5/dataset/valid/images/*.jpg')
print(len(valid_img_list))

/
495


In [ ]:
%cd /
from glob import glob
test_img_list = glob('/content/yolov5/dataset/test/images/*.jpg')
print(len(test_img_list))

/
264


In [ ]:
with open('/content/yolov5/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov5/dataset/val.txt', 'w') as f:
  f.write('\n'.join(valid_img_list) + '\n')

with open('/content/yolov5/dataset/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

In [ ]:
import yaml

with open('/content/yolov5/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/yolov5/dataset/train.txt'
data['val'] = '/content/yolov5/dataset/val.txt'
data['test'] =  '/content/yolov5/dataset/test.txt'

with open('/content/yolov5/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['green', 'red', 'yellow'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'traffic-light-oq7uj', 'url': 'https://universe.roboflow.com/traffic-light-detection-qsrxn/traffic-light-oq7uj/dataset/25', 'version': 25, 'workspace': 'traffic-light-detection-qsrxn'}, 'test': '../test/images', 'train': 'traffic-light-25/train/images', 'val': 'traffic-light-25/valid/images'}
{'names': ['green', 'red', 'yellow'], 'nc': 3, 'roboflow': {'license': 'CC BY 4.0', 'project': 'traffic-light-oq7uj', 'url': 'https://universe.roboflow.com/traffic-light-detection-qsrxn/traffic-light-oq7uj/dataset/25', 'version': 25, 'workspace': 'traffic-light-detection-qsrxn'}, 'test': '/content/yolov5/dataset/test.txt', 'train': '/content/yolov5/dataset/train.txt', 'val': '/content/yolov5/dataset/val.txt'}


In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 3 --data /content/yolov5/dataset/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name results

/content/yolov5
2024-03-08 05:22:28.487867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 05:22:28.487921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 05:22:28.489284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/yolov5/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weight

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/results3/weights/best.pt --img 416 --conf 0.5 --source ../s12.mp4